In [1]:
from Neuroradiomics.registration import registration_reader
from Neuroradiomics.registration import elastix_multimap_registration
from Neuroradiomics.registration import registration_writer
from Neuroradiomics.registration import apply_transform_from_files
from Neuroradiomics.registration import read_transform_from_files
from Neuroradiomics.registration import elastix_rigid_registration
from Neuroradiomics.registration import Set_sampler_parameters_as_image
from Neuroradiomics.registration import registration_transform_parameters_writer
from Neuroradiomics.registration import transform_parameters_writer
from Neuroradiomics.registration import evaluate_registration_mse
from Neuroradiomics.skull_stripping import negative_3d_masking



import itk
import numpy as np
import os
from itkwidgets import view

import tkinter as tk
from tkinter import filedialog

In [2]:
root = tk.Tk()
root.withdraw()


''

In [4]:
flair_filename = filedialog.askopenfilename(title = "Select the flair image")

    #Moving Image
t1_filename = filedialog.askopenfilename(title = "Select the t1 image")

atlas_filename = filedialog.askopenfilename(title = "Select the atlas image")

mask_filename = filedialog.askopenfilename(title = "Select the brain mask image")

flair, t1 = registration_reader(flair_filename, t1_filename)
atlas, mask = registration_reader(atlas_filename, mask_filename)


In [6]:
rigid_reg_obj = elastix_rigid_registration(flair, t1)
new_params = Set_sampler_parameters_as_image(rigid_reg_obj.GetTransformParameterObject(), t1)

reg_t1 = itk.transformix_filter(t1, new_params)                                

In [7]:
reg_atlas_obj = elastix_multimap_registration(reg_t1, atlas)

The Multimap Registration is done!


In [19]:
reg_mask = itk.transformix_filter(mask, reg_atlas_obj.GetTransformParameterObject())

In [20]:
masked_t1 = negative_3d_masking(reg_t1, reg_mask)

In [27]:
masked_reg_atlas = negative_3d_masking(reg_atlas_obj.GetOutput(), reg_mask)

In [28]:
reg_masked_t1 = elastix_multimap_registration(masked_t1, masked_reg_atlas)

The Multimap Registration is done!


In [29]:
output_filepath = filedialog.askdirectory(title = "Select where do you want to save the new image")
registration_writer(reg_masked_t1, output_filepath)

In [30]:
itk.imwrite(masked_reg_atlas, output_filepath+'/unregistered_masked_atlas.nii')

All your files are written


AttributeError: module 'itk' has no attribute 'imrite'

In [18]:
reg_atlas_obj.GetCurrentExactMetricValue()

AttributeError: 'itkElastixRegistrationMethodIF3IF3' object has no attribute 'GetCurrentExactMetricValue'